In [5]:
import cv2
import numpy as np

%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import os
import scipy
import math
import keras
from keras.models import load_model
from sklearn.externals import joblib

import PIL
import billiard
from billiard import Pool
from functools import partial
import pytesseract

In [6]:
def main():
    location = "../Text-detection-in-natural-scene-images-dataset/text or no text dataset/test images/"
    test_images = ['2750.jpg','74375.jpg','25541.jpg']
    
    mp_pool = billiard.Pool(processes=3)
    preprocess_mp_partial = partial(run_preprocessing, location=location)
    result_data = mp_pool.map(preprocess_mp_partial, test_images)
    
    #return result_data

    print(result_data)

In [7]:
def run_preprocessing(frame_location, location="."):
    single_frame_data_dict = {
        "text": "",
        "timestamp_ms": "0"
    }
    
    frame_abs_path = os.path.join(os.path.abspath(location), frame_location)
    frame_no = frame_abs_path.split("/")[-1].split(".")[0]
    img = cv2.imread(frame_abs_path, 0)
    
    thresold_frames = os.path.join(location, 'thresold_frames')
    binarized_frames = os.path.join(location, 'binarized_frames')
    
    if not os.path.exists(thresold_frames):
        os.makedirs(thresold_frames)
    if not os.path.exists(binarized_frames):
        os.makedirs(binarized_frames)
        
    
    #otsu_1 = cv2.threshold(img,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)[1]
    th2 = cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY,25,2)

    # connected components
    cc  = cv2.connectedComponents(np.uint8(th2))[1]

    # creating imgage for every single label in the connected components
    con_img = {}
    for label in np.unique(cc[cc>0]):
        image = np.zeros(cc.shape)
        indices = np.where(cc==label)
        image[indices] = 1
        con_img[label] = image

    tot1=0
    i=1
    
    loaded_scaler = joblib.load('./my_scaler_base_2.pkl')
    model = load_model('./my_model_base_2.h5')
    
    for elem in con_img.keys():
        x,y,w,h = cv2.boundingRect(np.uint8(con_img[elem]))
        #print("****", x, y, w, h)
        imng = img[y:y+h,x:x+w]
        imng =  scipy.misc.imresize(imng,(32,32)).ravel().reshape(1,1024)
        imng = loaded_scaler.transform(imng)
        imng = imng.reshape((1,32,32,1))
        if np.around(model.predict(imng)) == 1.0:
            tot1+=con_img[elem]
            cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),2)
            i+=1


    thresold_frame = Image.fromarray((cc * 255).astype(np.uint8))
    thresold_frame.save(os.path.join(thresold_frames, 'thresold_' + str(frame_no) + ".jpg"))
    
    binary_frame = Image.fromarray((tot1 * 255).astype(np.uint8))
    binary_frame.save(os.path.join(binarized_frames, 'binary_' + str(frame_no) + ".jpg"))
    #im.save("../Text-detection-in-natural-scene-images-dataset/text or no text dataset/test images/b.jpg")
    
    
    #final_image = cv2.imread("../Text-detection-in-natural-scene-images-dataset/text or no text dataset/test images/b.jpg", 0)
    final_image = cv2.imread(os.path.join(binarized_frames, 'binary_' + str(frame_no) + ".jpg"), 0)
    final_image = cv2.bitwise_not(final_image)
    text = pytesseract.image_to_string(final_image)
    
    single_frame_data_dict["text"] = text
    single_frame_data_dict["timestamp_ms"] = str(frame_no)

#     plt.figure(figsize=[20,10])
#     plt.subplot(1,3,1)
#     plt.imshow(img)

#     plt.subplot(1,3,2)
#     plt.imshow(cc)

#     plt.subplot(1,3,3)
#     plt.imshow(tot1, 'gray')
    
    return single_frame_data_dict

In [8]:
if __name__ == "__main__":
    main()

/home/void/Projects/OCR/Text-detection-in-natural-scene-images/lib/python3.6/site-packages/ipykernel_launcher.py:44: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
/home/void/Projects/OCR/Text-detection-in-natural-scene-images/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype uint8 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/home/void/Projects/OCR/Text-detection-in-natural-scene-images/lib/python3.6/site-packages/ipykernel_launcher.py:44: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
/home/void/Projects/OCR/Text-detection-in-natural-scene-images/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype uint8 was conver

[{'text': 'Should we men\nhelp our wives\ndo housework', 'timestamp_ms': '2750'}, {'text': 'Then one day\neveryt ng changed', 'timestamp_ms': '74375'}, {'text': 'He looked at me\n\nas if | had told him\n\n| was going to build\n\na space rocket\nhor', 'timestamp_ms': '25541'}]
